In [74]:
import torch
import os, io
import skimage.color
from torch import nn
import xml.etree.ElementTree as ET
from PIL import Image
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.optim as optim
from scipy.misc import imread
from collections import OrderedDict
import torchvision
import skimage
from torchvision import transforms, utils

In [158]:
import sys
print (sys.version)
print (torch.__version__)

3.6.4 (v3.6.4:d48eceb, Dec 19 2017, 06:54:40) [MSC v.1900 64 bit (AMD64)]
1.0.1


In [152]:
# torch.utils.data.DataLoader(dataset, batch_size=1, 
#                             shuffle=False, sampler=None, batch_sampler=None,
#                             num_workers=0, collate_fn=<function default_collate>, pin_memory=False, 
#                             drop_last=False, timeout=0, worker_init_fn=None)
class dataclass(Dataset):
    def __init__(self,allclasses,imagesDirectory,xmlDirectory,normalize = True,fivecrop=False,is330=False):
        f = open(allclasses,"r")
        self.labels_dict = {}
        self.number_labels = {}
        self.counter_labels = {}
        counter=0
        for line in f:
            self.labels_dict[line[:9]] = line[10:-1]
            self.number_labels[line[10:-1]] = counter
            self.counter_labels[counter] =line[10:-1] 
            counter+=1
        self.imageList = os.listdir(imagesDirectory)  # UNSAVED
        self.xmlsDirectory = os.listdir(xmlDirectory)  # UNSAVED
        if normalize and fivecrop:
            self.transforms =  transforms.Compose([transforms.ToPILImage(),transforms.Resize(280),torchvision.transforms.FiveCrop(224),
                                                   transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                                   transforms.Lambda(lambda norms: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(norm) for norm in norms]))])

        elif not normalize:
            self.transforms =  transforms.Compose([transforms.ToPILImage(),transforms.Resize(224),transforms.CenterCrop(224),transforms.ToTensor()])
        else:
            self.transforms =  transforms.Compose([transforms.ToPILImage(),transforms.Resize(224),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        if is330:
            self.transforms = transforms.Compose([transforms.ToPILImage(),transforms.Resize(380),torchvision.transforms.FiveCrop(330),
                                                   transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                                   transforms.Lambda(lambda norms: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(norm) for norm in norms]))])
    
    def __getitem__(self,idx):
        return_dict = self.retrieve_xml(self.xmlsDirectory[idx])
        #return_dict["image"] = io.imread("imagespart/"+ self.imageList[idx])
        holder = skimage.io.imread("imagespart/"+ self.imageList[idx])
        return_dict["class"] = self.number_labels[return_dict["name"]]
        if(len(holder.shape) == 2):
            holder = np.stack((holder,)*3, axis=-1)
        return_dict["image"] = holder
        return_dict["image"] = self.transforms(return_dict["image"])
#         print("return_dict['image'].shape")
#         print(return_dict["image"].shape)
        #return_dict["image"] = transforms.functional.to_grayscale(return_dict["image"], num_output_channels=1)
        
        
        #return_dict["image"] = self.auto_image_reshape(return_dict["image"],224,return_dict)
        
        #return_dict["image"] = return_dict["image"]
        #.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) ????
        #NORMALISE
        
        return return_dict
         
    def __len__(self):
            return len(self.imageList)        
        
    def retrieve_xml(self,reference):
        # take a single xml's name and obtain all information from it.
        root =ET.parse("val/" + reference).getroot()
        dicty = {}
        for sizevar in root.findall("size"):
            dicty["width"] = sizevar.findall("width")[0].text
            dicty["height"] = sizevar.findall("height")[0].text
            dicty["depth"] = sizevar.findall("depth")[0].text
        for obj in root.findall('object'):
            for name in obj.findall('name'):
#                 print(name.text)
                dicty["name"] = self.labels_dict[name.text]
            for pose in obj.findall('pose'):
                dicty["pose"] = pose.text
            for trunc in obj.findall('truncated'):
                dicty["trunc"]  = bool(int(trunc.text))
            for difficult in obj.findall('difficult'):
                dicty["difficult"] = bool(int(difficult.text))
            for bndbox in obj.findall('bndbox' ):
                dicty["xmin"] = int(bndbox.findall("xmin")[0].text)
                dicty["ymin"] = int(bndbox.findall("ymin")[0].text)
                dicty["xmax"] = int(bndbox.findall("xmax")[0].text)
                dicty["ymax"] = int(bndbox.findall("ymax")[0].text)
        return dicty

def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.eval()
    for batch_idx, dict_of_values in enumerate(train_loader):
        data = dict_of_values["image"]
        
        target = dict_of_values["class"]
        data, target = data.to(dict_of_values["image"]), target.to(dict_of_values["class"])
        optimizer.zero_grad()
        output = model(data)
#         print(output.shape)
#         print(target.shape)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return loss.item()

def validation_motion(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total=0
    occurrence = {}
    correctness ={}
    setcount=0
    with torch.no_grad():
        for dict_of_values in test_loader:
            data = dict_of_values["image"]
            target = dict_of_values["class"]
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            
            for kk in range(len(pred)):
                if pred[kk] == target[kk]:
                    correct+=1
                total+=1
            setcount+=1
            if setcount%10==0:
                print("Current Set: " + str(setcount))

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, correctness,occurrence
def validation_motion(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total=0
    occurrence = {}
    correctness ={}
    setcount=0
    with torch.no_grad():
        for dict_of_values in test_loader:
            data = dict_of_values["image"]
            target = dict_of_values["class"]
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            
            for kk in range(len(pred)):
                if pred[kk] == target[kk]:
                    correct+=1
                total+=1
            setcount+=1
            if setcount%10==0:
                print("Current Set: " + str(setcount))

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, correctness,occurrence

def validation_motion2(log_interval, model, device, test_loader,is5crop=False):
    model.eval()
    test_loss = 0
    correct = 0
    total=0
    occurrence = {}
    correctness ={}
    setcount=0
    with torch.no_grad():
        for dict_of_values in test_loader:
            data = dict_of_values["image"]
            target = dict_of_values["class"]
#             print("data shape: " + str(data.shape))
            if len(data.shape)>4:
                #target = target.view(-1,1).repeat(1,data.shape[1]).view(-1,1)
                data = data.view([-1,data.shape[-3],data.shape[-2],data.shape[-1]])
            data, target = data.to(device), target.to(device)
            output = model(data)
            if is5crop:
                output = output.view(test_loader.batch_size, 5, -1)
#                 print("output after view = "+ str(output.shape))
                pred = output.mean(dim=1) # get the index of the max log-probability
#                 print("output mean across dim 1 : "+ str(pred.shape))
                pred = pred.argmax(dim=1, keepdim=True).squeeze(1)
#                 print(pred.shape)
#                 print(target.shape)
            else:
                pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            
            for kk in range(len(pred)):
                if pred[kk] == target[kk]:
                    correct+=1
                total+=1
            setcount+=1
            if setcount%10==0:
                print("Current Set: " + str(setcount))

    test_loss = correct/total
    
    print('\n 5 crop Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, correctness,occurrence

In [154]:
class Squeeze330(nn.Module):
    def __init__(self, num_classes=1000):
        super(Squeeze330,self).__init__()
        self.num_classes = num_classes
        self.newPool = nn.AdaptiveAvgPool2d((224,224))
        self.reference = models.squeezenet1_1(pretrained = True)
    def forward(self,x):
        x = self.newPool(x)
        x = self.reference.features(x)
        x = self.reference.classifier(x)
        return x.view(x.size(0), self.num_classes)
    
class Inception330(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000):
        super(Inception330, self).__init__()
        self.newPool = nn.AdaptiveAvgPool2d((224,224))
        self.reference = models.inception_v3(pretrained=True)
    def forward(self, x):
        features = self.newPool(x)
        features = self.reference.forward(features)
        return features
    
def main():
    
    a = dataclass("synset_words.txt","imagespart/","val/",True,False,False) #with normalise
    b = dataclass("synset_words.txt","imagespart/","val/",False,False,False) #without normalise
    c = dataclass("synset_words.txt","imagespart/","val/",True,True,False) #normalised, fivecrop
    d = dataclass("synset_words.txt","imagespart/","val/",True,True,True) #normalised, fivecrop, 330 input
    image_dataload1 = DataLoader(a,batch_size=10,shuffle=True,num_workers =0,timeout=4)
    image_dataload2 = DataLoader(b,batch_size=10,shuffle=True,num_workers =0,timeout=4)
    image_dataload3 = DataLoader(c,batch_size=10,shuffle=True,num_workers =0,timeout=4)
    image_dataload4 = DataLoader(d,batch_size=10,shuffle=True,num_workers =0,timeout=4)
    device = torch.device("cpu")
    squeezenet = models.squeezenet1_1(pretrained = True)
    learning_rate = 0.01
    momentum_mod = 0.001
    optimizer = optim.SGD(squeezenet.parameters(), lr=learning_rate, momentum=momentum_mod)
    log_interval = 100
    print("without normalise")
    validation_motion(log_interval, squeezenet, device, image_dataload2)
    print("With normalise")
    validation_motion(log_interval, squeezenet, device, image_dataload1)
    print("5 crop with normalise")
    validation_motion2(log_interval, squeezenet, device, image_dataload3,True)
    print("squeezenet with 330")
    validation_motion2(log_interval, Squeeze330(), device, image_dataload4,True)
    print("Inception With 330")
    validation_motion2(log_interval, Inception330(), device, image_dataload4,True)
    
    
main()

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\torchvision\models\squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\torchvision\models\squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


without normalise
Current Set: 10
Current Set: 20
Current Set: 30
Current Set: 40
Current Set: 50
Current Set: 60
Current Set: 70
Current Set: 80
Current Set: 90
Current Set: 100
Current Set: 110
Current Set: 120
Current Set: 130
Current Set: 140
Current Set: 150
Current Set: 160
Current Set: 170
Current Set: 180
Current Set: 190
Current Set: 200
Current Set: 210
Current Set: 220
Current Set: 230
Current Set: 240
Current Set: 250

Test set: Average loss: 0.0000, Accuracy: 340/2500 (14%)

With normalise
Current Set: 10
Current Set: 20
Current Set: 30
Current Set: 40
Current Set: 50
Current Set: 60
Current Set: 70
Current Set: 80
Current Set: 90
Current Set: 100
Current Set: 110
Current Set: 120
Current Set: 130
Current Set: 140
Current Set: 150
Current Set: 160
Current Set: 170
Current Set: 180
Current Set: 190
Current Set: 200
Current Set: 210
Current Set: 220
Current Set: 230
Current Set: 240
Current Set: 250

Test set: Average loss: 0.0000, Accuracy: 1440/2500 (58%)

5 crop with norm

In [ ]:
        
#     def auto_image_reshape(self,image,target_smalls,boundcontainer):
#         # reshape your image and redefine boundaries.
#         if (image.shape[0]>target_smalls or image.shape[1]>target_smalls) and  (not image.shape[0]<=target_smalls and not image.shape[1]<target_smalls):
#             # if either is greater, and there is not a side that is <=target_smalls
#             if image.shape[0]<image.shape[1]:
# #                 print(image.shape)
#                 image_rescaled = transform.rescale(image,target_smalls/image.shape[0], anti_aliasing=False,multichannel = True)
#             else:
# #                 print(image.shape)
#                 image_rescaled = transform.rescale(image,target_smalls/image.shape[1], anti_aliasing=False,multichannel = True)
# #                 print(image_rescaled.shape)
#             #resize boundaries accordingly
#             boundcontainer["xmin"] = int(boundcontainer["xmin"]*image_rescaled.shape[0]/image.shape[0])
#             boundcontainer["xmax"] = int(boundcontainer["xmax"]*image_rescaled.shape[0]/image.shape[0])
#             boundcontainer["ymin"] = int(boundcontainer["ymin"]*image_rescaled.shape[1]/image.shape[1])
#             boundcontainer["ymax"] = int(boundcontainer["ymax"]*image_rescaled.shape[1]/image.shape[1])
#         else:
#             return image
#         return image_rescaled
    
#     def cropped_bounds(self,bounds,xmin,xmax,ymin,ymax):
#         # redefine bounds of label after a crop
#         return_xmin =0
#         return_xmax =0
#         return_ymin =0
#         return_ymax =0
#         if bounds["xmin"]<xmin:
#             return_xmin = bounds["xmin"]
#         else:
#             return_xmin = xmin
#         if bounds["xmax"]<xmax:
#             return_xmax = bounds["xmin"]
#         else:
#             return_xmax = xmax
#         if bounds["ymin"]<ymin:
#             return_ymin =bounds["ymin"]
#         else:
#             return_ymin = ymin
#         if bounds["ymax"]<ymax:
#             return_ymax =bounds["ymax"]
#         else:
#             return_ymax = ymax
#         return return_xmin,return_xmax,return_ymin,return_ymax
    
#     def five_crop(self,image,bounds):
#         #return all 5 crops, with bounds in a dictionary.
#         side1 = image.shape[0]
#         side2 = image.shape[1]
#         topleftdict = {}
#         toprightdict={}
#         botleftdict={}
#         botrightdict={}
#         centraldict = {}
#         #assume 3 channels in general.
#         topleft = plt.imshow(image[:int(side1/2),:int(side2/2),:])
#         xmin, xmax, ymin, ymax = cropped_bounds(bounds,0,int(side1/2),0, int(side2/2))
#         topleftdict["xmin"] = xmin
#         topleftdict["xmax"] = xmax
#         topleftdict["ymin"] = ymin
#         topleftdict["ymax"] = ymax
#         plt.show()
#         topright = plt.imshow(image[int(side1/2):,:int(side2/2),:])
#         xmin, xmax, ymin, ymax = cropped_bounds(bounds,int(side1/2),side1,0, int(side2/2))
#         toprightdict["xmin"] = xmin
#         toprightdict["xmax"] = xmax
#         toprightdict["ymin"] = ymin
#         toprightdict["ymax"] = ymax
#         plt.show()
#         botleft = plt.imshow(image[:int(side1/2),int(side2/2):,:])
#         xmin, xmax, ymin, ymax = cropped_bounds(bounds,0,int(side1/2),int(side2/2), side2)
#         botleftdict["xmin"] = xmin
#         botleftdict["xmax"] = xmax
#         botleftdict["ymin"] = ymin
#         botleftdict["ymax"] = ymax
#         plt.show()
#         botright = plt.imshow(image[int(side1/2):,int(side2/2):,:])
#         xmin, xmax, ymin, ymax = cropped_bounds(bounds,int(side1/2),side1,int(side2/2), side2)
#         botrightdict["xmin"] = xmin
#         botrightdict["xmax"] = xmax
#         botrightdict["ymin"] = ymin
#         botrightdict["ymax"] = ymax
#         plt.show()
#         central = plt.imshow(image[int(side1/4):int(side1* 3/4),int(side2/4):int(side2/4*3),:])
#         xmin, xmax, ymin, ymax = cropped_bounds(bounds,int(side1/4),int(side1*3/4),int(side2/4), int(side2/4*3))
#         centraldict["xmin"] = xmin
#         centraldict["xmax"] = xmax
#         centraldict["ymin"] = ymin
#         centraldict["ymax"] = ymax
#         plt.show()
#         return topleft,topleftdict,topright,toprightdict,botleft,botleftdict,botright,botrightdict,central,centraldict
        
        
    
    
#     def find_center(self,image):
#         # find the center of an image after it was resized to 224 and something. or something and 224
#         if image.shape[0]==224:
#             temp = int(image.shape[1]/2)
#             return_image = image[:,temp-112:temp+112,:]
#         else: #image.shape[1]==224
#             temp = int(image.shape[0]/2)
#             return_image = image[temp-112:temp+112,:,:]
# #         plt.imshow(return_image)
# #         print(return_image)
# #         plt.show()
#         return return_image


# def show_image_batch(sample_batched):
#     """Show image for a batch of samples."""
#     images_batch, xml_batch = sample_batched['image'], sample_batched['xml']
#     images_batch = torch.tensor(images_batch)
#     batch_size = len(images_batch)
#     im_size = images_batch.size(1)

#     grid = utils.make_grid(images_batch)
#     plt.imshow(grid.numpy().transpose((1, 2, 0)))
#     plt.title('Batch from dataloader')
   